In [19]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

import json
import pymongo as pm

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [58]:
## read in experimental metadata file (CSV)
 
## for example:
path_to_metadata = 'annotation_data.csv'
meta = pd.read_csv(path_to_metadata)
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append('"leg","arm","backrest","seat","pedestal"')
    if meta.category[i]=="dog":
        parts.append('"eye","mouth","ear","body","tail","neck","leg"')
    if meta.category[i] == "bird":
        parts.append('"beak","wings","feet","tail","body"')
    if meta.category[i] == "car":
        parts.append('"wheel", "windshield", "window", "door", "bumper", "lamp"')
parts = pd.Series(parts)
meta['parts']= parts
meta['svg'] =meta['svg'].str.replace('u', '')
meta['svg'] =meta['svg'].str.replace("'",'"')

        

,target,svg,category,parts
0,knob,"[""M69,167c0,27.13435 5,54.84972 5,81c0,1.81504...",chair,"""leg"",""arm"",""backrest"",""seat"",""pedestal"""
1,redsport,"[""M51,224c0,-65.25556 75.24654,-35.24654 83,-4...",car,"""wheel"", ""windshield"", ""window"", ""door"", ""bump..."
2,bluesport,"[""M34,212c5.3353,-10.6706 6.56215,-22.81075 9,...",car,"""wheel"", ""windshield"", ""window"", ""door"", ""bump..."
3,chihuahua,"[""M99,157c-26.4479,17.63193 -2.9121,54.60807 1...",dog,"""eye"",""mouth"",""ear"",""body"",""tail"",""neck"",""leg"""
4,pug,"[""M69,172c0,4.72338 -5.37528,22.62472 -2,26c9....",dog,"""eye"",""mouth"",""ear"",""body"",""tail"",""neck"",""leg"""
5,weimaraner,"[""M31,149c31.51565,0 63.69848,1.39154 95,4c27....",dog,"""eye"",""mouth"",""ear"",""body"",""tail"",""neck"",""leg"""
6,redantique,"[""M39,203c-1.61473,-12.9178 -11.98219,-32.5089...",car,"""wheel"", ""windshield"", ""window"", ""door"", ""bump..."
7,basset,"[""M136,123c-10.03489,0 -22.63418,-2.12194 -32,...",dog,"""eye"",""mouth"",""ear"",""body"",""tail"",""neck"",""leg"""
8,waiting,"[""M65,288c0,-47.26597 -10.61179,-88.33968 29,-...",chair,"""leg"",""arm"",""backrest"",""seat"",""pedestal"""
9,inlay,"[""M53,198c5.4241,-32.54462 25.86878,-21.08688 ...",chair,"""leg"",""arm"",""backrest"",""seat"",""pedestal"""


In [61]:
## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)
    
J = json.loads(open('annotation_meta.js',mode='rU').read())
assert len(J)==len(meta)

##'{} unique games.'.format(len(np.unique(meta2.gameID.values)))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 'U' mode is deprecated
  # Remove the CWD from sys.path while we load stuff.


### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [3]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [4]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_demo']

In [10]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('data.json',mode='ru').read())

In [11]:
## actually add data now to the database
for (i,j) in enumerate(J):
    if i%1==0:
        print ('%d of %d' % (i,len(J)))
    coll.insert_one(j)

0 of 4
1 of 4
2 of 4
3 of 4


In [12]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))

We have 4 sketches.
We have these kinds: [u'smiley', u'house', u'chair', u'bird']
